#### Домашнее задание.
## Урок 6 - Множества и опрации над ними
#### Выполнил работу: Данил Кузнецов
Вам будет выдано две таблицы (два файла):
1. Таблица с событиями покупок пользователей
a. user_id — id пользователя,
b. event_time — время покупки (DateTime),
c. revenue — сумма покупки (USD).
2. Таблица с событиями и датами установки/привлечения пользователей
a. user_id — id пользователя,
b. install_date — дата установки (Date).

## На основании этих двух таблиц вам нужно будет построить несколько когортных метрик.
● User install — количество установок в этот день (по сути, размер когорты).
● Paying Users — количество платящих (заплативших хотя бы раз) пользователей,
установивших игру в этот день.
● Revenue — выручка от всех пользователей, установивших игру в этот день.
● ARPU (average revenue per user) — средний чек на пользователя.
○ считается по формуле ARPU = Revenue/Users Installs.
● ARPPU (average revenue per paying user) — средний чек платящего пользователя.
○ считается как ARPPU = Revenue/Paying users.


In [1]:
import pandas as pd

In [2]:
installs = pd.read_csv('installs.csv',index_col = 0)
installs.head()


,client_id,install_date
0,4324237485864989747,2021-01-29
1,2614649573193308458,2021-01-29
2,5216305460009860018,2021-01-29
3,9005903977217811684,2021-01-29
4,8800689291083556692,2021-01-29


In [3]:
payments = pd.read_csv('payments.csv')
payments.head()

,Unnamed: 0,event_timestamp,client_id,revenue,purchase_type,is_tester,is_youtuber
0,351,2021-01-01,4660245999561852056,853.0,pack,0,0
1,439,2021-01-01,4905393702707182978,277.0,hard,0,0
2,472,2021-01-01,613720643259957998,2151.0,pack,0,0
3,498,2021-01-02,5041691475431075286,1153.0,hard,0,0
4,516,2021-01-02,1334726919424607014,1616.0,hard,0,0


In [4]:
user_install = installs.groupby(['install_date']).agg({'client_id':'nunique'})
user_install.head()

,client_id
install_date,
2021-01-01,779
2021-01-02,993
2021-01-03,1063
2021-01-04,1115
2021-01-05,1065


In [5]:
data = installs.merge(payments, on='client_id', how='right') 
data.head()

,client_id,install_date,Unnamed: 0,event_timestamp,revenue,purchase_type,is_tester,is_youtuber
0,4660245999561852056,2021-01-01,351,2021-01-01,853.0,pack,0,0
1,4905393702707182978,2021-01-01,439,2021-01-01,277.0,hard,0,0
2,613720643259957998,2021-01-01,472,2021-01-01,2151.0,pack,0,0
3,5041691475431075286,2021-01-01,498,2021-01-02,1153.0,hard,0,0
4,1334726919424607014,2021-01-02,516,2021-01-02,1616.0,hard,0,0


In [6]:
paying_users_and_revenue = data.groupby(['install_date']).agg({'client_id':'nunique','revenue':'sum'})
paying_users_and_revenue.head()

,client_id,revenue
install_date,,
2021-01-01,31,43235.0
2021-01-02,42,145236.0
2021-01-03,49,112920.0
2021-01-04,32,74144.0
2021-01-05,39,45698.0


In [7]:
final_table = user_install.merge(paying_users_and_revenue, on='install_date', how='inner')



In [10]:
final_table.columns = ['user_install','paying_users','revenue']

In [11]:
final_table['revenue'] = final_table['revenue'].astype('int')

final_table.head()

,user_install,paying_users,revenue
install_date,,,
2021-01-01,779,31,43235
2021-01-02,993,42,145236
2021-01-03,1063,49,112920
2021-01-04,1115,32,74144
2021-01-05,1065,39,45698


In [16]:
final_table['arpu'] = round(final_table['revenue']/final_table['user_install'], 1)

final_table['arppu'] = round(final_table['revenue']/final_table['paying_users'], 1)

final_table

,user_install,paying_users,revenue,arpu,arppu
install_date,,,,,
2021-01-01,779,31,43235,55.5,1394.7
2021-01-02,993,42,145236,146.3,3458.0
2021-01-03,1063,49,112920,106.2,2304.5
2021-01-04,1115,32,74144,66.5,2317.0
2021-01-05,1065,39,45698,42.9,1171.7
2021-01-06,1029,31,52275,50.8,1686.3
2021-01-07,998,46,51869,52.0,1127.6
2021-01-08,1051,30,48225,45.9,1607.5
2021-01-09,1029,35,60553,58.8,1730.1
